# Riconoscimento Linee

Ora che abbiamo calibrato la camera possiamo dedicarci al primo e fondamentale passo per un algoritmo di guida autonoma: il **riconoscimento delle linee stradali**.

La città di CheemsCity è caratterizzata da strade nere con linee bianche a segnalarne i bordi e tutte le analisi che andremo a fare adesso si baseranno su questo assunto;  
qual'ora il vostro robot viaggiasse in una città con regole diverse vi basterà cambiare poche cose.

## Matrice Omografica

La **matrice omografica** è un particolare tipo di matrice che ci permette di passare facilmente da un sistema di riferimento ad un altro, ed è molto utile per descrivere rototraslazioni.   
Noi andremo a calcolare la matrice che codifica il passaggio dal sistema di riferimento della foto a quello del mondo reale (da 2D a 3D), in particolare il centro del nuovo sistema di  
riferimento si troverà nella proiezione sul terreno del centro della fotocamera.

Per poter calcolare la matrice vi occorrerà il foglio con la scacchiera che dovreste aver stampato nel punto precedente.

In [ ]:
import cv2
import time
import numpy as np
import os
import yaml
import glob
import matplotlib.pyplot as plt

In [ ]:
from picamera import PiCamera

Andiamo ora ad importare la configurazione che avevamo effettuato prima.

In [ ]:
file = open("part1/FinalCalibration.yml", "r")
calibration_data = yaml.load(file, Loader=yaml.UnsafeLoader)
matrix = calibration_data['camera_matrix']
dist_coef = calibration_data['distortion_coefficient']

andiamo ora a calcolare una mappa che permetterà di correggere gli errori della camera in modo veloce.

In [ ]:
#TO-DO: inserire dimensioni dell'immagine o fare programma che calcola.
mapx, mapy = cv2.initUndistortRectifyMap(
                cam_matrix, dist_coeff, None, cam_matrix, (w, h), 5)

creiamo ora una funzione che ci permetterà di sfruttare questa mappa per la correzzione delle immagini

In [ ]:
def undistort_faster(image, mapx, mapy):
    try:
        cv2.remap(image, mapx, mapy, cv2.INTER_LINEAR)
        return True
    else:
        return False

Posizioniamo la scaccchiera a terra e mettiamo la camera nell'apposito spazio. Una volta posizionata andiamo a definire delle variabili che ne indicheranno la distanza dal punto zero dello scacchiera.

1. **offsety**: distanza lungo l'asse parallelo al lato più vicino della scacchiera.
2. **offsetx**: distanza camera-scacchiera, asse perpendicolare a questa

In [ ]:
camera_calibration_square_size: 0.018
offsety = 3 * camera_calibration_square_size
offsetx = 0.102 
board_offset = np.array([offsetx, -offsety])
nx = #nx
ny = #ny

E ora come nel caso precedente della calibrazione della camera, sfrutteremo le foto per il calcolo della matrice omogenea. Il vantaggio è che in questo caso ce ne servirà solo una. Il primo blocco è per chi utilizza una picamera, il secondo per quelle USB.

In [ ]:
import picamera
#programma per picamera raspberry pi

camera = Picamera()
print("avvio camera")
camera.start_preview()
print("foto tra 4 secondi")
time.sleep(4)
photo_name = "part2/foto/omografia.jpg"
camera.capture(photo_name)
camera.stop_preview()

#programma per camera nativa o USB

cam = cv2.VideoCapture(0)
while True:
    print("avvio camera")
    ret, image = cam.read()
	cv2.imshow('photo',image)
	k = cv2.waitKey(1)
    if k == ord(s):
        photo_name = "part2/foto/omografia.jpg"
        cv2.imwrite(photo_name, image)
    if k == ord(q):
		break

controlliamo ora che la foto scattata sia valida per la calibrazione, in caso contrario rirunna le celle precedenti.

image = glob.glob('part2/foto/omografia.jpg')

for fname in image:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (nx,ny), None)
    if ret == True:
        total = total + 1
print("numero di foto buone: {}".format(total))

Definiamo e cerchiamo i punti necessari per la calibrazione, saranno gli stessi del foglio precedente.

In [ ]:
src_pts = []
for r in range(ny):
    for c in range(nx):
        src_pts.append(
            np.array([r * square_size, c * square_size],
                        dtype='float32') + board_offset)

src_pts.reverse()

imgpoints = []

In [ ]:
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

ret, corners = cv.findChessboardCorners(gray, (nx,ny), None)

if ret == True:
    corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
    imgpoints.append(corners2)

In [ ]:
 H, _mask = cv2.findHomography(
            chessboard.imgpoints.reshape(len(chessboard.imgpoints), 2),
            np.array(src_pts), cv2.RANSAC)

In [ ]:
calibration_data = {
    calibration_data = {
            "H_matrix": H,
}
with open('part2/Homography.yml', 'w') as outfile:
    yaml.dump(calibration_data, outfile, default_flow_style = False)